In [1]:
from counterfactuals.explainers import Fimap
from counterfactuals.visualization import compare
from counterfactuals.constraints import ValueMonotonicity, ValueNominal, Freeze
from data import AdultData
from tensorflow import keras
import random
import numpy as np

import warnings
warnings.filterwarnings('ignore')

random.seed(10)
np.random.seed(10)


In [2]:
adult_data = AdultData('data/datasets/adult.csv')
model = keras.models.load_model('models/model_adult')
model_predictions = model.predict(adult_data.X_train)
model_predictions[model_predictions > 0.5] = 1
model_predictions[model_predictions <= 0.5] = 0



In [3]:
fimap = Fimap()
fimap.fit(adult_data.X_train, model_predictions, epochs=1)

test_X = adult_data.X_test.iloc[0]
cf = fimap.generate(test_X)

[]

Training s
Training loss (for one batch) at step 0: 0.2928 
Training accuracy 0.80659896
Training loss (for one batch) at step 0: 0.1509 
Training accuracy 0.8669816
{0: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add')>, 1: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_1')>, 2: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_2')>, 3: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_3')>, 4: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_4')>, 5: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_5')>, 6: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_6')>, 7: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_7')>, 8: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_8')>, 9: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_9')>, 10: <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_

In [ ]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
relationship,0.000000,1.932193,1.932193
marital.status,2.000000,3.062429,1.062429
occupation,5.000000,7.326011,2.326011
workclass,3.000000,3.432882,0.432882
native.country,38.000000,38.241962,0.241962
race,4.000000,3.921935,-0.078065
sex,1.000000,0.811261,-0.188739
education,15.000000,11.450332,-3.549668
age,-0.702455,0.296849,0.999304
capital.loss,-0.216660,0.308004,0.524664


In [ ]:
for constraint in adult_data.constraints:
    print(constraint)
    
fimap = Fimap(constraints=[ValueNominal(["sex"]), Freeze(["sex"])], l1=0.1, l2=0.1)
fimap.fit(adult_data.X_train, model_predictions, epochs=10)

test_X = adult_data.X_test.iloc[0]
cf = fimap.generate(test_X)

ValueNominal(columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country'], values=[])
Freeze(columns=['race', 'sex', 'native.country'])


AttributeError: 'DataFrameMapper' object has no attribute '_continuous_columns'

In [ ]:
fimap._constraints

[]

In [ ]:
# compare(test_X, cf.iloc[0])

cf

In [ ]:
test_X

In [ ]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(adult_data.X_train.columns), bces=[fimap], dist=2, h=8, lambda_=0.2, n_jobs=1)

cf2 = ece.generate(adult_data.X_test.iloc[0])


In [ ]:
compare(adult_data.X_test.iloc[0], cf2.iloc[0])